# Data Cleaning and Pre-processing

## Imports

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import time
from tqdm import tqdm
import pickle as pkl

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

In [4]:
root_url ='https://www.carspecs.us'

In [5]:
# refise rear and front suspension fearures no 54

## Initial Data Cleaning - Eliminating Duplicates, Creating Uniform Classes and Transforming Numbers to Floats and Integers

In [6]:
df_full = pkl.load(open('df_pickles/df_full.p','rb'))

In [7]:
df = df_full.copy()

In [8]:
df.shape[0]-736

2280

### Duplicates

In [9]:
df[df.index.duplicated(keep=False)].shape

(1472, 102)

In [10]:
df = df.loc[~df.index.duplicated(keep='first')]

In [11]:
columns_tobe_dropped_1 = ['Gross weight', 'Spare Wheel Diameter', 'EPA MPG Equivalent - City', 
                          'EPA MPG Equivalent - Hwy', 'Valves', 'Cargo capacity, all seats in place', 
                          'Maximum cargo capacity', 'Curb Weight Rear', 'Curb Weight Front', 
                          'Cargo Area Length @ Floor to Seat 2', 'Cargo Area Length @ Floor to Seat 1', 
                          'Liftover Height', 'Battery Range', 'EPA interior volume']
columns_tobe_dropped_2 = ['Aux Fuel Tank Location', 'Fuel Tank Location', 
                          'Cargo Box Width @ Wheelhousings','Ground clearance']
df.drop(columns_tobe_dropped_1+columns_tobe_dropped_2, axis=1, inplace=True)

In [14]:
# sort dataframe according to number of nans
import operator
dict_sorted = sorted(dict(df.isna().sum()).items(), key=operator.itemgetter(1), reverse=True)
df_null = df.isna().unstack()
t = df_null[df_null]
t;
s = pd.Series(t.index.get_level_values(1), t.index.get_level_values(0))
pd.DataFrame(s.groupby(level=0).apply(list));

In [15]:
dict_sorted;

### Creating Uniformity among features and converting numbers into int/floats

In [18]:
df.iloc[100:200,0:20]

,0-60 mph,3rd Row Head Room,3rd Row Hip Room,3rd Row Leg Room,3rd Row Shoulder Room,4th Row Head Room,4th Row Hip Room,4th Row Leg Room,4th Row Shoulder Room,ABS Brakes Type,ABS Braking System (second line),Base engine size,Brake Type,Curb weight,Cylinders,Dead Weight Hitch - Max Tongue Weight,Dead Weight Hitch - Max Trailer Weight,Drive type,EPA MPG Equivalent - Combined,EPA mileage (combined)
/cars/2019/audi/a5/79114,5.20 sec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-Wheel,NaN,3.00 L,ABS Brakes,"3,847 lbs.",6,NaN,NaN,All Wheel Drive,NaN,24 mpg
/cars/2019/audi/a5/79113,5.57 sec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-Wheel,NaN,3.00 L,ABS Brakes,"4,211 lbs.",6,NaN,NaN,All Wheel Drive,NaN,24 mpg
/cars/2019/audi/a5/79115,5.57 sec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-Wheel,NaN,3.00 L,ABS Brakes,"4,211 lbs.",6,NaN,NaN,All Wheel Drive,NaN,24 mpg
/cars/2019/audi/a5-sportback/77699,5.67 sec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-Wheel,NaN,2.00 L,ABS Brakes,"3,704 lbs.",4,NaN,NaN,All Wheel Drive,NaN,NaN
/cars/2019/audi/a5-sportback/77698,5.67 sec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-Wheel,NaN,2.00 L,ABS Brakes,"3,704 lbs.",4,NaN,NaN,All Wheel Drive,NaN,NaN
/cars/2019/audi/a5-sportback/77700,5.67 sec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-Wheel,NaN,2.00 L,ABS Brakes,"3,704 lbs.",4,NaN,NaN,All Wheel Drive,NaN,NaN
/cars/2019/audi/a6/77707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-Wheel,NaN,3.00 L,ABS Brakes,NaN,6,NaN,NaN,All Wheel Drive,NaN,NaN
/cars/2019/audi/a6/77708,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-Wheel,NaN,3.00 L,ABS Brakes,NaN,6,NaN,NaN,All Wheel Drive,NaN,NaN
/cars/2019/audi/a6/77709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-Wheel,NaN,3.00 L,ABS Brakes,NaN,6,NaN,NaN,All Wheel Drive,NaN,NaN
/cars/2019/audi/a7/77711,5.53 sec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-Wheel,NaN,3.00 L,ABS Brakes,"4,001 lbs.",6,NaN,NaN,All Wheel Drive,NaN,NaN


In [19]:
df.iloc[:,19].value_counts().sum(), df.iloc[:,19].isna().sum() # (1342, 1419)

(1342, 938)

In [20]:
df.at['/cars/2019/acura/ilx/79088', '0-60 mph']

'6.63 sec'

In [21]:
for col in [list(df.columns)[0]]+[list(df.columns)[11]]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            item_v = float(re.findall('\d+.\d+',df.at[i, col])[0])
            df.at[i, col] = item_v

# column names = [0-60 mph, Base engine size]

0-60 mph
Base engine size


In [22]:
for col in list(df.columns)[1:9]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            item_v = float(re.findall('\d+.\d+',df.at[i, col])[0])
            df.at[i, col] = item_v

# column names = [3rd Row Head Room, 3rd Row Hip Room, 3rd Row Leg Room, 3rd Row Shoulder Room, 4th Row Head Room, 4th Row Hip Room, 4th Row Leg Room, 4th Row Shoulder Room]

3rd Row Head Room
3rd Row Hip Room
3rd Row Leg Room
3rd Row Shoulder Room
4th Row Head Room
4th Row Hip Room
4th Row Leg Room
4th Row Shoulder Room


In [23]:
for col in [list(df.columns)[9]]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            if df.at[i, col] == 'Yes':
                item_v = '4-Wheel'
                df.at[i, col] = item_v
                
# column names = [ABS Brakes Type]

ABS Brakes Type


In [24]:
for col in [list(df.columns)[10]]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            if df.at[i, col] == 'NA':
                item_v = np.nan
                df.at[i, col] = item_v
# [ABS Braking System (second line)]

ABS Braking System (second line)


In [25]:
for col in [list(df.columns)[13]]+list(df.columns)[15:17]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            item_v = float("".join(re.findall("\d+",df.at[i, col])))
            df.at[i, col] = item_v
# [Curb weight, Dead Weight Hitch - Max Tongue Weight, Dead Weight Hitch - Max Trailer Weight]

Curb weight
Dead Weight Hitch - Max Tongue Weight
Dead Weight Hitch - Max Trailer Weight


In [26]:
for col in [list(df.columns)[14]]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            item_v = int(df.at[i, col])
            df.at[i, col] = item_v
            
#[Cylinders]

Cylinders


In [27]:
for col in [list(df.columns)[17]]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            item_v = (df.at[i, col]).lower()
            df.at[i, col] = item_v
            
# [Drive type]

Drive type


In [28]:
for col in list(df.columns)[18:20]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            item_v = int(re.findall('\d+',df.at[i, col])[0])
            df.at[i, col] = item_v
            
# [EPA MPG Equivalent - Combined, EPA mileage (combined)]

EPA MPG Equivalent - Combined
EPA mileage (combined)


In [29]:
for i in df.index:
    if (type(df.at[i, 'EPA mileage (combined)']) == type(np.nan)) and (type(df.at[i,'EPA MPG Equivalent - Combined']) != type(np.nan)):
        item_v = df.at[i,'EPA MPG Equivalent - Combined']
        df.at[i, 'EPA mileage (combined)'] = item_v

In [30]:
df.iloc[100:200,20:40]

,Eighth Gear Ratio,Engine Oil Cooler,Engine type,Fifth Gear Ratio,Final Drive Axle Ratio,First Gear Ratio,Fourth Gear Ratio,Front Axle Type,Front Brake Rotor Diameter,Front Brakes,Front Suspension Type,Front Tire Size,Front Wheel Diameter,Front Wheel Material,Front Wheel Width,Front head room,Front hip room,Front leg room,Front shoulder room,Front track
/cars/2019/audi/a5/79114,0.67,NaN,Gas,1.28,2.85,4.71,1.67,NaN,13.80 in.,Front Disc Brakes,Multi-Link,P245/40YR18,18.00 in.,Aluminum,8.50 in.,39.00 in.,NaN,41.30 in.,55.30 in.,62.50 in.
/cars/2019/audi/a5/79113,0.67,NaN,Gas,1.28,2.85,4.71,1.67,NaN,13.80 in.,Front Disc Brakes,Multi-Link,P245/40YR18,18.00 in.,Aluminum,8.50 in.,40.00 in.,NaN,41.30 in.,55.30 in.,62.50 in.
/cars/2019/audi/a5/79115,0.67,NaN,Gas,1.28,2.85,4.71,1.67,NaN,13.80 in.,Front Disc Brakes,Multi-Link,P245/40YR18,18.00 in.,Aluminum,8.50 in.,40.00 in.,NaN,41.30 in.,55.30 in.,62.50 in.
/cars/2019/audi/a5-sportback/77699,NaN,NaN,Gas,0.74,4.27,3.19,1.06,NaN,13.30 in.,Front Disc Brakes,Multi-Link,P245/40HR18,18.00 in.,Aluminum,8.50 in.,39.40 in.,NaN,41.30 in.,55.70 in.,62.50 in.
/cars/2019/audi/a5-sportback/77698,NaN,NaN,Gas,0.74,4.27,3.19,1.06,NaN,13.30 in.,Front Disc Brakes,Multi-Link,P245/40HR18,18.00 in.,Aluminum,8.50 in.,39.40 in.,NaN,41.30 in.,55.70 in.,62.50 in.
/cars/2019/audi/a5-sportback/77700,NaN,NaN,Gas,0.74,4.27,3.19,1.06,NaN,13.30 in.,Front Disc Brakes,Multi-Link,P245/40HR18,18.00 in.,Aluminum,8.50 in.,39.40 in.,NaN,41.30 in.,55.70 in.,62.50 in.
/cars/2019/audi/a6/77707,NaN,NaN,Gas,0.74,4.41,3.19,1.06,NaN,14.80 in.,Front Disc Brakes,Multi-Link,P245/45HR19,19.00 in.,Aluminum,8.00 in.,NaN,NaN,NaN,NaN,64.20 in.
/cars/2019/audi/a6/77708,NaN,NaN,Gas,0.74,4.41,3.19,1.06,NaN,14.80 in.,Front Disc Brakes,Multi-Link,P245/45HR19,19.00 in.,Aluminum,8.00 in.,NaN,NaN,NaN,NaN,64.20 in.
/cars/2019/audi/a6/77709,NaN,NaN,Gas,0.74,4.41,3.19,1.06,NaN,14.80 in.,Front Disc Brakes,Multi-Link,P245/45HR19,19.00 in.,Aluminum,8.00 in.,NaN,NaN,NaN,NaN,64.20 in.
/cars/2019/audi/a7/77711,NaN,NaN,Gas,0.74,4.41,3.19,1.06,NaN,14.80 in.,Front Disc Brakes,Multi-Link,P245/45HR19,19.00 in.,Aluminum,8.00 in.,NaN,NaN,NaN,NaN,65.00 in.


In [31]:
df.iloc[:,30].value_counts()

Strut                           1107
Double Wishbone                  608
Multi-Link                       134
Short And Long Arm               125
MacPherson Strut                  91
Independent                       57
Other                             33
- TBD -                           28
Torsion Bar                       20
Coil-Over Shock                   16
Leading Link                       7
Independent MacPherson Strut       7
Coil Spring                        6
HiPer Strut                        5
Double Wishbone Pushrod            3
Magnetic Ride Control              2
Name: Front Suspension Type, dtype: int64

In [32]:
for col in [list(df.columns)[20]]+list(df.columns)[23:27]+[list(df.columns)[28]]+[list(df.columns)[32]]+list(df.columns)[34:40]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            item_v = float(re.findall('\d+.\d+',df.at[i, col])[0])
            df.at[i, col] = item_v
            
# [Eighth Gear Ratio
# Fifth Gear Ratio,
# Final Drive Axle Ratio,
# First Gear Ratio,
# Fourth Gear Ratio,
# Front Brake Rotor Diameter,
# Front Wheel Diameter,
# Front Wheel Width,
# Front head room,
# Front hip room,
# Front leg room,
# Front shoulder room,
# Front track]

Eighth Gear Ratio
Fifth Gear Ratio
Final Drive Axle Ratio
First Gear Ratio
Fourth Gear Ratio
Front Brake Rotor Diameter
Front Wheel Diameter
Front Wheel Width
Front head room
Front hip room
Front leg room
Front shoulder room
Front track


In [33]:
for col in [list(df.columns)[21]]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            if df.at[i, col] == 'Regular Duty' or df.at[i, col] =='HD' or df.at[i, col] =='Yes':
                item_v = 'yes'
                df.at[i, col] = item_v
            elif df.at[i, col] == 'NA' or df.at[i, col] =='None':
                item_v = np.nan
                df.at[i, col] = item_v
# [Engine Oil Cooler]

Engine Oil Cooler


In [34]:
for col in [list(df.columns)[27]]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            if df.at[i, col] == 'Independent Driving':
                item_v = 'Independent'
                df.at[i, col] = item_v
            elif df.at[i, col] == '- TBD -':
                item_v = 'Other'
                df.at[i, col] = item_v

# [Front Axle Type]

Front Axle Type


In [35]:
for col in [list(df.columns)[30]]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            if df.at[i, col] == 'MacPherson Strut' or df.at[i, col] == 'Strut':
                item_v = 'Independent MacPherson Strut'
                df.at[i, col] = item_v
            elif df.at[i, col] == 'Coil Spring':
                item_v = 'Coil-Over Shock'
                df.at[i, col] = item_v
                
# [Front Suspension Type]

Front Suspension Type


In [36]:
for col in [list(df.columns)[33]]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            if df.at[i, col] == 'Forged Aluminum':
                item_v = 'Aluminum'
                df.at[i, col] = item_v
            elif df.at[i, col] == 'Styled Steel':
                item_v = 'Steel'
                df.at[i, col] = item_v
                
# [Front Wheel Material]

Front Wheel Material


In [37]:
df.iloc[100:200,40:60]

,Fuel tank capacity,Gross Vehicle Weight Rating Cap,Height,Horsepower,Horsepower RPM,Length,Max Trailering Capacity,Ninth Gear Ratio,Passenger Capacity,Passenger Doors,Rear Axle Type,Rear Brake Rotor Diameter,Rear Brakes,Rear Drum Brake Dimensions,Rear Suspension Type,Rear Tire Size,Rear Wheel Diameter,Rear Wheel Material,Rear Wheel Width,Rear head room
/cars/2019/audi/a5/79114,15.30 gal.,NaN,53.90 in.,349 hp,"5,400",184.70 in.,NaN,NaN,4,2,NaN,13.00 in.,Rear Disc Brakes,NA in.,Multi-Link,P245/40YR18,18.00 in.,Aluminum,8.50 in.,36.10 in.
/cars/2019/audi/a5/79113,15.30 gal.,NaN,54.40 in.,349 hp,"5,400",184.70 in.,NaN,NaN,4,2,NaN,13.00 in.,Rear Disc Brakes,NA in.,Multi-Link,P245/40YR18,18.00 in.,Aluminum,8.50 in.,36.30 in.
/cars/2019/audi/a5/79115,15.30 gal.,NaN,54.40 in.,349 hp,"5,400",184.70 in.,NaN,NaN,4,2,NaN,13.00 in.,Rear Disc Brakes,NA in.,Multi-Link,P245/40YR18,18.00 in.,Aluminum,8.50 in.,36.30 in.
/cars/2019/audi/a5-sportback/77699,15.30 gal.,NaN,54.60 in.,252 hp,"5,000",186.30 in.,NaN,NaN,5,4,NaN,13.00 in.,Rear Disc Brakes,NA in.,Multi-Link,P245/40HR18,18.00 in.,Aluminum,8.50 in.,NaN
/cars/2019/audi/a5-sportback/77698,15.30 gal.,NaN,54.60 in.,252 hp,"5,000",186.30 in.,NaN,NaN,5,4,NaN,13.00 in.,Rear Disc Brakes,NA in.,Multi-Link,P245/40HR18,18.00 in.,Aluminum,8.50 in.,NaN
/cars/2019/audi/a5-sportback/77700,15.30 gal.,NaN,54.60 in.,252 hp,"5,000",186.30 in.,NaN,NaN,5,4,NaN,13.00 in.,Rear Disc Brakes,NA in.,Multi-Link,P245/40HR18,18.00 in.,Aluminum,8.50 in.,NaN
/cars/2019/audi/a6/77707,NaN,NaN,57.40 in.,335 hp,"5,000",194.50 in.,NaN,NaN,5,4,NaN,13.80 in.,Rear Disc Brakes,NA in.,Multi-Link,P245/45HR19,19.00 in.,Aluminum,8.00 in.,NaN
/cars/2019/audi/a6/77708,NaN,NaN,57.40 in.,335 hp,"5,000",194.50 in.,NaN,NaN,5,4,NaN,13.80 in.,Rear Disc Brakes,NA in.,Multi-Link,P245/45HR19,19.00 in.,Aluminum,8.00 in.,NaN
/cars/2019/audi/a6/77709,NaN,NaN,57.40 in.,335 hp,"5,000",194.50 in.,NaN,NaN,5,4,NaN,13.80 in.,Rear Disc Brakes,NA in.,Multi-Link,P245/45HR19,19.00 in.,Aluminum,8.00 in.,NaN
/cars/2019/audi/a7/77711,NaN,NaN,56.00 in.,335 hp,"5,000",195.60 in.,NaN,NaN,5,4,NaN,13.80 in.,Rear Disc Brakes,NA in.,Multi-Link,P245/45HR19,19.00 in.,Aluminum,8.00 in.,NaN


In [38]:
df.iloc[:,54].value_counts()

Multi-Link                 1366
Leaf                        273
Torsion Beam                177
Double Wishbone             176
5-Link                       35
- TBD -                      35
2-Stage Multi-Leaf           33
Strut                        28
Trailing Arm                 25
4-Link                       21
Multi-Leaf                   20
Five-Link                    12
Independent Multi-Link        8
Short And Long Arm            7
Independent                   6
Hypoid Drive                  6
Other                         5
Compound Crank                5
Double Wishbone Pushrod       3
Magnetic Ride Control         2
Solid Axle                    2
Trailing Link                 2
Independent Five-Link         2
Name: Rear Suspension Type, dtype: int64

In [39]:
for col in [list(df.columns)[40]]+[list(df.columns)[42]]+[list(df.columns)[45]]+[list(df.columns)[47]]+[list(df.columns)[51]]+[list(df.columns)[56]]+list(df.columns)[58:60]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            item_v = float(re.findall('\d+.\d+',df.at[i, col])[0])
            df.at[i, col] = item_v
# [Fuel tank capacity,
# Height,
# Length,
# Ninth Gear Ratio,
# Rear Brake Rotor Diameter,
# Rear Wheel Diameter,
# Rear Wheel Width,
# Rear head room]

Fuel tank capacity
Height
Length
Ninth Gear Ratio
Rear Brake Rotor Diameter
Rear Wheel Diameter
Rear Wheel Width
Rear head room


In [40]:
for col in list(df.columns)[43:45]+[list(df.columns)[41]]+[list(df.columns)[46]]+list(df.columns)[48:50]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            if len(df.at[i, col]) >1:
                item_v = float("".join(re.findall("\d+",df.at[i, col])))
                df.at[i, col] = item_v
            elif len(df.at[i, col]) ==1:
                item_v = int(df.at[i, col])
                df.at[i, col] = item_v
                
# [Horsepower,
# Horsepower RPM,
# Gross Vehicle Weight Rating Cap,
# Max Trailering Capacity,
# Passenger Capacity,
# Passenger Doors]

Horsepower
Horsepower RPM
Gross Vehicle Weight Rating Cap
Max Trailering Capacity
Passenger Capacity
Passenger Doors


In [41]:
for col in [list(df.columns)[50]]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            if df.at[i, col] == 'Semi Floating':
                item_v = 'Semi-Floating'
                df.at[i, col] = item_v
            elif df.at[i, col] == 'Styled Steel':
                item_v = 'Steel'
                df.at[i, col] = item_v
            elif df.at[i, col] == 'Semi-floating' :
                item_v = 'Semi-Floating'
                df.at[i, col] = item_v
            elif df.at[i, col] == 'Solid axle w/five-link location' :
                item_v = 'Solid Axle w/Five-Link Location'
                df.at[i, col] = item_v
                
# [Rear Axle Type]

Rear Axle Type


In [42]:
for col in [list(df.columns)[57]]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            if df.at[i, col] == 'Forged Aluminum':
                item_v = 'Aluminum'
                df.at[i, col] = item_v
            elif df.at[i, col] == 'Styled Steel':
                item_v = 'Steel'
                df.at[i, col] = item_v
                
# [Rear Wheel Material]

Rear Wheel Material


In [43]:
for col in [list(df.columns)[54]]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            if (df.at[i, col] == 'Five-Link') or (df.at[i, col] == '5-Link') or (df.at[i, col] == '4-Link') or (df.at[i, col] == 'Multi-Link') or (df.at[i, col] == 'Independent Five-Link'):
                item_v = 'Independent Multi-Link'
                df.at[i, col] = item_v
            elif (df.at[i, col] == 'Leaf') or (df.at[i, col] == '2-Stage Multi-Leaf'):
                item_v = 'Multi-Leaf'
                df.at[i, col] = item_v
            elif (df.at[i, col] == 'Short And Long Arm') or (df.at[i, col] == 'Double Wishbone Pushrod'):
                item_v = 'Double Wishbone'
                df.at[i, col] = item_v
            elif (df.at[i, col] == 'Trailing Link'):
                item_v = 'Trailing Arm'
                df.at[i, col] = item_v
            elif (df.at[i, col] == '- TBD -') or (df.at[i, col] == 'Independent'):
                item_v = 'Other'
                df.at[i, col] = item_v




# [Rear Suspension Type]

Rear Suspension Type


In [44]:
df.iloc[100:200,60:80]

,Rear hip room,Rear leg room,Rear shoulder room,Rear track,Reverse Ratio,Second Gear Ratio,Seventh Gear Ratio,Sixth Gear Ratio,Spare Tire Size,Spare Wheel Material,TPMS,Third Gear Ratio,Torque,Torque RPM,Transmission,Turning circle,Weight Distributing Hitch - Max Tongue Weight,Weight Distributing Hitch - Max Trailer Weight,Wheelbase,Width
/cars/2019/audi/a5/79114,NaN,32.70 in.,50.70 in.,61.70 in.,3.32,3.14,0.84,1.00,Compact,Steel,Tire Pressure Monitor,2.11,369 ft-lbs.,"1,370",Transmission: 8-Speed Tiptronic Automatic,37.70 ft.,NaN,NaN,108.80 in.,72.70
/cars/2019/audi/a5/79113,NaN,33.10 in.,46.90 in.,61.70 in.,3.32,3.14,0.84,1.00,Compact,Steel,Tire Pressure Monitor,2.11,369 ft-lbs.,"1,370",Transmission: 8-Speed Tiptronic Automatic -inc...,37.70 ft.,NaN,NaN,108.90 in.,72.70
/cars/2019/audi/a5/79115,NaN,33.10 in.,46.90 in.,61.70 in.,3.32,3.14,0.84,1.00,Compact,Steel,Tire Pressure Monitor,2.11,369 ft-lbs.,"1,370",Transmission: 8-Speed Tiptronic Automatic -inc...,37.70 ft.,NaN,NaN,108.90 in.,72.70
/cars/2019/audi/a5-sportback/77699,NaN,35.10 in.,54.50 in.,61.70 in.,2.75,2.19,0.39,0.51,Compact,Steel,NaN,1.52,273 ft-lbs.,"1,600",Transmission: 7-Speed S tronic Dual-Clutch Aut...,38.40 ft.,NaN,NaN,111.20 in.,72.60
/cars/2019/audi/a5-sportback/77698,NaN,35.10 in.,54.50 in.,61.70 in.,2.75,2.19,0.39,0.51,Compact,Steel,NaN,1.52,273 ft-lbs.,"1,600",Transmission: 7-Speed S tronic Dual-Clutch Aut...,38.40 ft.,NaN,NaN,111.20 in.,72.60
/cars/2019/audi/a5-sportback/77700,NaN,35.10 in.,54.50 in.,61.70 in.,2.75,2.19,0.39,0.51,Compact,Steel,NaN,1.52,273 ft-lbs.,"1,600",Transmission: 7-Speed S tronic Dual-Clutch Aut...,38.40 ft.,NaN,NaN,111.20 in.,72.60
/cars/2019/audi/a6/77707,NaN,NaN,NaN,63.70 in.,2.75,2.19,0.39,0.51,Compact,Steel,Tire Pressure Monitor,1.52,369 ft-lbs.,"1,370",Transmission: 7-Speed Stronic,40.00 ft.,NaN,NaN,115.10 in.,74.30
/cars/2019/audi/a6/77708,NaN,NaN,NaN,63.70 in.,2.75,2.19,0.39,0.51,Compact,Steel,Tire Pressure Monitor,1.52,369 ft-lbs.,"1,370",Transmission: 7-Speed Stronic,40.00 ft.,NaN,NaN,115.10 in.,74.30
/cars/2019/audi/a6/77709,NaN,NaN,NaN,63.70 in.,2.75,2.19,0.39,0.51,Compact,Steel,Tire Pressure Monitor,1.52,369 ft-lbs.,"1,370",Transmission: 7-Speed Stronic,40.00 ft.,NaN,NaN,115.10 in.,74.30
/cars/2019/audi/a7/77711,NaN,NaN,NaN,64.40 in.,2.75,2.19,0.39,0.51,Compact,Steel,Tire Pressure Monitor,1.52,369 ft-lbs.,"1,370",Transmission: 7-Speed Automatic w/S tronic,40.00 ft.,NaN,NaN,115.20 in.,75.10


In [45]:
df.iloc[:,79].value_counts();

In [46]:
for col in list(df.columns)[60:68]+[list(df.columns)[71]]+[list(df.columns)[75]]+list(df.columns)[78:80]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            item_v = float(re.findall('\d+.\d+',df.at[i, col])[0])
            df.at[i, col] = item_v
            
# [Rear hip room
# Rear leg room,
# Rear shoulder room,
# Rear track,
# Reverse Ratio,
# Second Gear Ratio,
# Seventh Gear Ratio,
# Sixth Gear Ratio,
# Third Gear Ratio,
# Turning circle,
# Wheelbase,
# Width]

Rear hip room
Rear leg room
Rear shoulder room
Rear track
Reverse Ratio
Second Gear Ratio
Seventh Gear Ratio
Sixth Gear Ratio
Third Gear Ratio
Turning circle
Wheelbase
Width


In [47]:
for col in list(df.columns)[72:74]+list(df.columns)[76:78]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            if len(df.at[i, col]) >1:
                item_v = float("".join(re.findall("\d+",df.at[i, col])))
                df.at[i, col] = item_v
            elif len(df.at[i, col]) ==1:
                item_v = int(df.at[i, col])
                df.at[i, col] = item_v
                
# [Torque,
# Torque RPM,
# Weight Distributing Hitch - Max Tongue Weight,
# Weight Distributing Hitch - Max Trailer Weight]

Torque
Torque RPM
Weight Distributing Hitch - Max Tongue Weight
Weight Distributing Hitch - Max Trailer Weight


In [48]:
df.iloc[200:230,80:]

,extra options,extra_options,price,model_name_carspecs.us
/cars/2019/bmw/6-series/77776,NaN,\n\n\nABS And Driveline Traction Control\nActi...,70300,2019 BMW 6-Series 640i xDrive Gran Turismo
/cars/2019/bmw/6-series/77777,NaN,\n\n\nABS And Driveline Traction Control\nAirb...,93100,2019 BMW 6-Series 650i Gran Coupe
/cars/2019/bmw/6-series/77778,NaN,\n\n\nABS And Driveline Traction Control\nAirb...,96100,2019 BMW 6-Series 650i xDrive Gran Coupe
/cars/2019/bmw/6-series/77779,NaN,\n\n\nABS And Driveline Traction Control\nAirb...,124400,2019 BMW 6-Series ALPINA B6 xDrive Gran Coupe
/cars/2019/bmw/7-series/77780,NaN,\n\n\nABS And Driveline Traction Control\nActi...,91250,2019 BMW 7-Series 740e xDrive iPerformance Plu...
/cars/2019/bmw/7-series/77781,NaN,\n\n\nABS And Driveline Traction Control\nActi...,83650,2019 BMW 7-Series 740i Sedan
/cars/2019/bmw/7-series/77782,NaN,\n\n\nABS And Driveline Traction Control\nActi...,86650,2019 BMW 7-Series 740i xDrive Sedan
/cars/2019/bmw/7-series/77783,NaN,\n\n\nABS And Driveline Traction Control\nActi...,96950,2019 BMW 7-Series 750i Sedan
/cars/2019/bmw/7-series/77784,NaN,\n\n\nABS And Driveline Traction Control\nActi...,99950,2019 BMW 7-Series 750i xDrive Sedan
/cars/2019/bmw/7-series/77785,NaN,\n\n\nABS And Driveline Traction Control\nActi...,139350,2019 BMW 7-Series ALPINA B7 xDrive Sedan


In [49]:
type(df.iloc[70,82])

int

In [50]:
for col in list(df.columns)[80:82]:
    print(col)
    for i in df[col].index:
        if type(df.at[i, col]) != type(np.nan):
            item_v = (df.at[i, col]).strip().split('\n')
            df.at[i, col] = item_v
            
# [extra options ,extra_options]

extra options
extra_options


In [51]:
for i in df.index:
    if (type(df.at[i, 'extra_options']) == type(np.nan)) and (type(df.at[i,'extra options']) != type(np.nan)):
        item_v = df.at[i,'extra options']
        df.at[i, 'extra_options'] = item_v

In [52]:
pkl.dump(df, open('df_pickles/df_semi-cleaned.p','wb'))

## Handling NaN/Missing Data

In [53]:
df_reg = pkl.load(open('df_pickles/df_semi-cleaned.p','rb'))

In [54]:
columns_tobe_dropped_3 = ['EPA MPG Equivalent - Combined', 'Rear Drum Brake Dimensions', 'extra options']

In [55]:
df_reg.columns

Index(['0-60 mph', '3rd Row Head Room', '3rd Row Hip Room', '3rd Row Leg Room',
       '3rd Row Shoulder Room', '4th Row Head Room', '4th Row Hip Room',
       '4th Row Leg Room', '4th Row Shoulder Room', 'ABS Brakes Type',
       'ABS Braking System (second line)', 'Base engine size', 'Brake Type',
       'Curb weight', 'Cylinders', 'Dead Weight Hitch - Max Tongue Weight',
       'Dead Weight Hitch - Max Trailer Weight', 'Drive type',
       'EPA MPG Equivalent - Combined', 'EPA mileage (combined)',
       'Eighth Gear Ratio', 'Engine Oil Cooler', 'Engine type',
       'Fifth Gear Ratio', 'Final Drive Axle Ratio', 'First Gear Ratio',
       'Fourth Gear Ratio', 'Front Axle Type', 'Front Brake Rotor Diameter',
       'Front Brakes', 'Front Suspension Type', 'Front Tire Size',
       'Front Wheel Diameter', 'Front Wheel Material', 'Front Wheel Width',
       'Front head room', 'Front hip room', 'Front leg room',
       'Front shoulder room', 'Front track', 'Fuel tank capacity',
       '

In [56]:
df_reg.isna().sum()[0:10]

0-60 mph                 1019
3rd Row Head Room        1959
3rd Row Hip Room         1973
3rd Row Leg Room         1959
3rd Row Shoulder Room    1959
4th Row Head Room        2274
4th Row Hip Room         2274
4th Row Leg Room         2274
4th Row Shoulder Room    2274
ABS Brakes Type            31
dtype: int64

### Fourth row seating

In [57]:
 df_reg[(df_reg['4th Row Head Room'] > 0  )][['4th Row Head Room', '4th Row Hip Room', '4th Row Leg Room', '4th Row Shoulder Room', 'Passenger Capacity']]

,4th Row Head Room,4th Row Hip Room,4th Row Leg Room,4th Row Shoulder Room,Passenger Capacity
/cars/2019/gmc/savana-passenger/78340,37.6,65.7,36.6,69.1,12
/cars/2019/gmc/savana-passenger/78341,37.6,65.7,36.6,69.1,12
/cars/2019/gmc/savana-passenger/78342,37.6,65.7,36.6,69.1,12
/cars/2019/gmc/savana-passenger/78343,37.6,65.7,36.6,69.1,12
/cars/2019/gmc/savana-passenger/78344,37.6,65.7,36.6,69.1,12
/cars/2019/gmc/savana-passenger/78345,37.6,65.7,36.6,69.1,12


In [58]:
 df_reg[(df_reg['4th Row Head Room'] > 0  )][['4th Row Head Room', '4th Row Hip Room', '4th Row Leg Room', '4th Row Shoulder Room', 'Passenger Capacity']].shape

(6, 5)

In [59]:
 df_reg[(df_reg['3rd Row Head Room'] > 0  )][['3rd Row Head Room', '3rd Row Hip Room', '3rd Row Leg Room', '3rd Row Shoulder Room', 'Passenger Capacity']]

,3rd Row Head Room,3rd Row Hip Room,3rd Row Leg Room,3rd Row Shoulder Room,Passenger Capacity
/cars/2019/acura/mdx/77624,35.6,40.6,28.1,54.7,7
/cars/2019/acura/mdx/77621,35.6,40.6,28.1,54.7,6
/cars/2019/acura/mdx/77620,35.6,40.6,28.1,54.7,7
/cars/2019/acura/mdx/77623,35.6,40.6,28.1,54.7,7
/cars/2019/acura/mdx/77622,35.6,40.6,28.1,54.7,6
/cars/2019/acura/mdx/77619,35.6,40.6,28.1,54.7,7
/cars/2019/acura/mdx/77625,35.6,40.6,28.1,54.7,6
/cars/2019/acura/mdx/77626,35.6,40.6,28.1,54.7,7
/cars/2019/acura/mdx/77628,35.6,40.6,28.1,54.7,6
/cars/2019/acura/mdx/77627,35.6,40.6,28.1,54.7,7


In [60]:
df_reg.drop([ '4th Row Head Room', '4th Row Hip Room', '4th Row Leg Room', '4th Row Shoulder Room', 
             '3rd Row Hip Room'], axis=1, inplace=True)

### Third row seating

In [61]:
df_reg['3rd Row Head Room'].fillna(value =0 , inplace=True)
df_reg['3rd Row Leg Room'].fillna(value =0 , inplace=True)
df_reg['3rd Row Shoulder Room'].fillna(value =0 , inplace=True)

In [62]:
# df_reg.ix[(df_reg['3rd Row Hip Room'].isna()) & (df_reg['3rd Row Leg Room'] == 0), '3rd Row Hip Room'] = df_reg.ix[(df_reg['3rd Row Hip Room'].isna()) & (df_reg['3rd Row Leg Room'] == 0), '3rd Row Hip Room'].fillna(value =0)

In [63]:
# Creating and interaction factor by multiplying thrid seat attributes
df_reg['third_row_seating'] = (df_reg['3rd Row Head Room']* df_reg['3rd Row Leg Room']* df_reg['3rd Row Shoulder Room'])
df_reg['third_row_seating']

/cars/2019/acura/ilx/79088         0.000
/cars/2019/acura/ilx/79090         0.000
/cars/2019/acura/ilx/79089         0.000
/cars/2019/acura/ilx/79092         0.000
/cars/2019/acura/ilx/79091         0.000
/cars/2019/acura/mdx/77624     54719.692
/cars/2019/acura/mdx/77621     54719.692
/cars/2019/acura/mdx/77620     54719.692
/cars/2019/acura/mdx/77623     54719.692
/cars/2019/acura/mdx/77622     54719.692
/cars/2019/acura/mdx/77619     54719.692
/cars/2019/acura/mdx/77625     54719.692
/cars/2019/acura/mdx/77626     54719.692
/cars/2019/acura/mdx/77628     54719.692
/cars/2019/acura/mdx/77627     54719.692
/cars/2019/acura/mdx/77631     54719.692
/cars/2019/acura/mdx/77629     54719.692
/cars/2019/acura/mdx/77630     54719.692
/cars/2019/acura/nsx/79093         0.000
/cars/2019/acura/rdx/77635         0.000
/cars/2019/acura/rdx/77633         0.000
/cars/2019/acura/rdx/77632         0.000
/cars/2019/acura/rdx/77634         0.000
/cars/2019/acura/rdx/77639         0.000
/cars/2019/acura

In [64]:
df_reg.drop(df_reg[['3rd Row Head Room', '3rd Row Leg Room', '3rd Row Shoulder Room']],axis=1,inplace=True)

### ABS brake types

In [65]:
df_reg['ABS Brakes Type'].fillna(method='bfill', inplace=True)

In [66]:
df_reg.isna().sum()[0:10]
# Base engine size, Curb weight, Cylinders, Drive type, EPA mileage (combined)

0-60 mph                                  1019
ABS Brakes Type                              0
ABS Braking System (second line)          1785
Base engine size                            66
Brake Type                                  33
Curb weight                                715
Cylinders                                   67
Dead Weight Hitch - Max Tongue Weight     1232
Dead Weight Hitch - Max Trailer Weight    1232
Drive type                                   1
dtype: int64

### Drive type

In [67]:
df_reg['Drive type'].fillna(method='bfill', inplace=True)

### Weights

In [68]:
df_reg['Curb weight'].value_counts()

3858.0    13
3418.0    11
4134.0    11
3990.0    10
4398.0     9
3450.0     9
4649.0     8
3891.0     8
2844.0     8
3490.0     8
3840.0     8
3455.0     7
3704.0     7
3660.0     7
3627.0     7
4330.0     7
3956.0     7
4394.0     7
3285.0     7
3530.0     7
3428.0     6
3362.0     6
4387.0     6
4784.0     6
3373.0     6
4101.0     6
3062.0     6
3327.0     6
3541.0     6
3900.0     6
          ..
3512.0     1
3822.0     1
3820.0     1
3815.0     1
2494.0     1
3488.0     1
3487.0     1
3484.0     1
3860.0     1
5170.0     1
2847.0     1
3869.0     1
2337.0     1
2416.0     1
3454.0     1
2432.0     1
4483.0     1
3461.0     1
4489.0     1
4507.0     1
3468.0     1
3470.0     1
8591.0     1
3473.0     1
3475.0     1
3934.0     1
3477.0     1
4502.0     1
4503.0     1
4209.0     1
Name: Curb weight, Length: 731, dtype: int64

In [69]:
df_reg['Dead Weight Hitch - Max Tongue Weight'].fillna(value=0,inplace=True)
df_reg['Dead Weight Hitch - Max Trailer Weight'].fillna(value=0,inplace=True)

In [71]:
df_reg['Dead Weight Hitch - Max Trailer Weight'].fillna(value=0,inplace=True)

### EPA mileage combined

In [72]:
for i in df_reg.index:
    if (type(df_reg.at[i, 'EPA mileage (combined)']) == type(np.nan)) and 
    (type(df_reg.at[i,'EPA MPG Equivalent - Combined']) != type(np.nan)):
        
        item_v = df_reg.at[i,'EPA MPG Equivalent - Combined']
        df_reg.at[i, 'EPA mileage (combined)'] = item_v

In [73]:
df_reg[(df_reg['EPA MPG Equivalent - Combined'].notna())][['EPA MPG Equivalent - Combined',
                                                           'EPA mileage (combined)']]

,EPA MPG Equivalent - Combined,EPA mileage (combined)
/cars/2019/bmw/5-series/77766,72,72
/cars/2019/bmw/5-series/77767,67,67
/cars/2019/bmw/7-series/77780,64,64
/cars/2019/bmw/i3/79144,113,113
/cars/2019/bmw/i3/79143,31,100
/cars/2019/bmw/i3/79146,113,113
/cars/2019/bmw/i3/79145,31,100
/cars/2019/chrysler/pacifica/78071,84,84
/cars/2019/chrysler/pacifica/78072,84,84
/cars/2019/chrysler/pacifica/78073,84,84


In [74]:
df_reg.drop(['EPA MPG Equivalent - Combined'],1,inplace=True)

### Gear Ratios

In [75]:
list(df_reg[(df_reg['First Gear Ratio'].isna())][['Eighth Gear Ratio','Seventh Gear Ratio', 
                                                  'Sixth Gear Ratio', 'Fifth Gear Ratio', 'Fourth Gear Ratio',
                                                  'Third Gear Ratio', 'Second Gear Ratio', 
                                                  'First Gear Ratio']].index)

['/cars/2019/bentley/bentayga/79130',
 '/cars/2019/bentley/bentayga/79132',
 '/cars/2019/bmw/i3/79144',
 '/cars/2019/bmw/i3/79143',
 '/cars/2019/bmw/i3/79146',
 '/cars/2019/bmw/i3/79145',
 '/cars/2019/buick/encore/77820',
 '/cars/2019/buick/encore/77823',
 '/cars/2019/buick/envision/70046',
 '/cars/2019/buick/envision/70047',
 '/cars/2019/buick/envision/70049',
 '/cars/2019/buick/envision/70048',
 '/cars/2019/buick/envision/70052',
 '/cars/2019/buick/envision/70050',
 '/cars/2019/buick/envision/70051',
 '/cars/2019/buick/lacrosse/77825',
 '/cars/2019/buick/lacrosse/77826',
 '/cars/2019/buick/lacrosse/77827',
 '/cars/2019/buick/lacrosse/77831',
 '/cars/2019/buick/lacrosse/77833',
 '/cars/2019/buick/regal/79155',
 '/cars/2019/buick/regal/79156',
 '/cars/2019/buick/regal/79157',
 '/cars/2019/buick/regal/79158',
 '/cars/2019/buick/regal/79159',
 '/cars/2019/buick/regal/79160',
 '/cars/2019/buick/regal/79161',
 '/cars/2019/buick/regal/79162',
 '/cars/2019/buick/regal/79163',
 '/cars/2019/bu

In [76]:
df_reg_scrape_missing = df_reg[df_reg['First Gear Ratio'].isna()][['Eighth Gear Ratio','Seventh Gear Ratio', 
                                                                   'Sixth Gear Ratio', 'Fifth Gear Ratio', 
                                                                   'Fourth Gear Ratio', 'Third Gear Ratio', 
                                                                   'Second Gear Ratio', 'First Gear Ratio']]

In [77]:
def soups_(url):
    r = requests.get(url)
    print(r.status_code)
    return BeautifulSoup(r.content, 'html.parser')

In [78]:
new_urls_gear_ratios=[]
for i,r in df_reg_scrape_missing.iterrows():
    new_urls_gear_ratios.append(i.replace('2019','2017')[:-5])

In [79]:
new_urls_gear_ratios[0:3]

['/cars/2017/bentley/bentayga/',
 '/cars/2017/bentley/bentayga/',
 '/cars/2017/bmw/i3/']

In [80]:
# list1 =[]
# for url in new_urls_gear_ratios:
#     dict1={}
#     for item in soups_(root_url+url).findAll('div',{'class':"pure-u-1 pure-u-md-1-2"}):
#         try:
#             if (item.find('h4').text) == 'First Gear Ratio':
#                 text = item.text.strip().split('\r\n')
#                 dict1[text[0]]= float(text[1])
#             if (item.find('h4').text) == 'Second Gear Ratio':
#                 text = item.text.strip().split('\r\n')
#                 dict1[text[0]]= float(text[1])
#             if (item.find('h4').text) == 'Third Gear Ratio':
#                 text = item.text.strip().split('\r\n')
#                 dict1[text[0]]= float(text[1])
#             if (item.find('h4').text) == 'Fourth Gear Ratio':
#                 text = item.text.strip().split('\r\n')
#                 dict1[text[0]]= float(text[1])
#             if (item.find('h4').text) == 'Fifth Gear Ratio':
#                 text = item.text.strip().split('\r\n')
#                 dict1[text[0]]= float(text[1])
#             if (item.find('h4').text) == 'Sixth Gear Ratio':
#                 text = item.text.strip().split('\r\n')
#                 dict1[text[0]]= float(text[1])
#             if (item.find('h4').text) == 'Seventh Gear Ratio':
#                 text = item.text.strip().split('\r\n')
#                 dict1[text[0]]= float(text[1])
#             if (item.find('h4').text) == 'Eighth Gear Ratio':
#                 text = item.text.strip().split('\r\n')
#                 dict1[text[0]]= float(text[1])
#         except:
#             pass

#     if dict1:
#         list1.append({url:dict1})
#     else:
#         dict1={}
#         new_url_2018 = url.replace('2017','2018')
#         for item in soups_(root_url+new_url_2018).findAll('div',{'class':"pure-u-1 pure-u-md-1-2"}):
#             try:
#                 if (item.find('h4').text) == 'First Gear Ratio':
#                     text = item.text.strip().split('\r\n')
#                     dict1[text[0]]= float(text[1])
#                 if (item.find('h4').text) == 'Second Gear Ratio':
#                     text = item.text.strip().split('\r\n')
#                     dict1[text[0]]= float(text[1])
#                 if (item.find('h4').text) == 'Third Gear Ratio':
#                     text = item.text.strip().split('\r\n')
#                     dict1[text[0]]= float(text[1])
#                 if (item.find('h4').text) == 'Fourth Gear Ratio':
#                     text = item.text.strip().split('\r\n')
#                     dict1[text[0]]= float(text[1])
#                 if (item.find('h4').text) == 'Fifth Gear Ratio':
#                     text = item.text.strip().split('\r\n')
#                     dict1[text[0]]= float(text[1])
#                 if (item.find('h4').text) == 'Sixth Gear Ratio':
#                     text = item.text.strip().split('\r\n')
#                     dict1[text[0]]= float(text[1])
#                 if (item.find('h4').text) == 'Seventh Gear Ratio':
#                     text = item.text.strip().split('\r\n')
#                     dict1[text[0]]= float(text[1])
#                 if (item.find('h4').text) == 'Eighth Gear Ratio':
#                     text = item.text.strip().split('\r\n')
#                     dict1[text[0]]= float(text[1])
#             except:
#                 pass
#         if dict1:
#             list1.append({new_url_2018:dict1})
#         else:
#             dict1={}
#             new_url_2016 = url.replace('2018','2016')
#             for item in soups_(root_url+new_url_2018).findAll('div',{'class':"pure-u-1 pure-u-md-1-2"}):
#                 try:
#                     if (item.find('h4').text) == 'First Gear Ratio':
#                         text = item.text.strip().split('\r\n')
#                         dict1[text[0]]= float(text[1])
#                     if (item.find('h4').text) == 'Second Gear Ratio':
#                         text = item.text.strip().split('\r\n')
#                         dict1[text[0]]= float(text[1])
#                     if (item.find('h4').text) == 'Third Gear Ratio':
#                         text = item.text.strip().split('\r\n')
#                         dict1[text[0]]= float(text[1])
#                     if (item.find('h4').text) == 'Fourth Gear Ratio':
#                         text = item.text.strip().split('\r\n')
#                         dict1[text[0]]= float(text[1])
#                     if (item.find('h4').text) == 'Fifth Gear Ratio':
#                         text = item.text.strip().split('\r\n')
#                         dict1[text[0]]= float(text[1])
#                     if (item.find('h4').text) == 'Sixth Gear Ratio':
#                         text = item.text.strip().split('\r\n')
#                         dict1[text[0]]= float(text[1])
#                     if (item.find('h4').text) == 'Seventh Gear Ratio':
#                         text = item.text.strip().split('\r\n')
#                         dict1[text[0]]= float(text[1])
#                     if (item.find('h4').text) == 'Eighth Gear Ratio':
#                         text = item.text.strip().split('\r\n')
#                         dict1[text[0]]= float(text[1])
#                 except:
#                     pass
#             if dict1:
#                 list1.append({new_url_2016:dict1})
#             else:
#                 dict1['First Gear Ratio']= np.nan
#                 dict1['Second Gear Ratio']= np.nan
#                 dict1['Third Gear Ratio']= np.nan
#                 dict1['Fourth Gear Ratio']= np.nan
#                 dict1['Fifth Gear Ratio']= np.nan
#                 dict1['Sixth Gear Ratio']= np.nan
#                 dict1['Seventh Gear Ratio']= np.nan
#                 dict1['Eighth Gear Ratio']= np.nan
#             list1.append({new_url_2016:dict1})

In [81]:
# pkl.dump(list1, open('webscrape_b/kbb_scrape_links', 'wb'))

In [82]:
list1 = pkl.load(open('webscrape_b/kbb_scrape_links', 'rb'))

In [83]:
df_ratios = pd.concat(list(map(lambda x: pd.DataFrame(x).T, list1)))

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [84]:
df_ratios.shape

(436, 8)

In [85]:
len(new_urls_gear_ratios)

436

In [86]:
df_ratios.index = df_reg_scrape_missing.index

In [87]:
df_ratios['First Gear Ratio'].isna().sum()

275

In [88]:
df_reg.loc[df_reg.index.isin(df_ratios.index), ['Eighth Gear Ratio','Seventh Gear Ratio', 'Sixth Gear Ratio', 
                                                'Fifth Gear Ratio', 'Fourth Gear Ratio', 'Third Gear Ratio', 
                                                'Second Gear Ratio', 'First Gear Ratio']] = df_ratios[['Eighth Gear Ratio','Seventh Gear Ratio', 'Sixth Gear Ratio', 
                                                                                                       'Fifth Gear Ratio', 'Fourth Gear Ratio', 'Third Gear Ratio', 'Second Gear Ratio', 
                                                                                                       'First Gear Ratio']]


In [89]:
df_reg.loc['/cars/2019/bentley/bentayga/79130','Eighth Gear Ratio']

0.67

In [90]:
df_reg[df_reg['First Gear Ratio'].isna()][['Eighth Gear Ratio','Seventh Gear Ratio', 
                                           'Sixth Gear Ratio', 'Fifth Gear Ratio', 'Fourth Gear Ratio', 
                                           'Third Gear Ratio', 'Second Gear Ratio', 'First Gear Ratio']].shape

(275, 8)

In [91]:
df_reg.isna().sum()[23:35]

Front Wheel Width                   275
Front head room                     218
Front hip room                      646
Front leg room                      204
Front shoulder room                 215
Front track                         553
Fuel tank capacity                   68
Gross Vehicle Weight Rating Cap    1688
Height                               38
Horsepower                           32
Horsepower RPM                       66
Length                              803
dtype: int64

### Front row seating

In [93]:
# interaction feature for front seat
df_apply = df_reg[(df_reg['Front head room'] > 0  )][['Front head room', 'Front leg room','Front shoulder room']]
df_reg['first_row_seating'] = df_apply.apply(lambda row: row['Front head room']*row['Front leg room']*row['Front shoulder room'], axis=1)

In [94]:
df_reg['first_row_seating'].isna().sum()

261

In [95]:
df_reg.drop(['Front head room', 'Front leg room','Front shoulder room', 
             'Front hip room','Rear Axle Type'],1,inplace=True)

### Rear row seating

In [96]:
# interaction feature for rear seat
df_apply_rear = df_reg[(df_reg['Rear head room'] > 0  )][['Rear head room', 'Rear leg room','Rear shoulder room']]
df_reg['rear_row_seating'] = df_apply_rear.apply(lambda row: row['Rear head room']*row['Rear leg room']*
                                                 row['Rear shoulder room'], axis=1)

In [97]:
df_reg.drop(['Rear head room', 'Rear leg room','Rear shoulder room', 'Rear hip room'],1,inplace=True)

In [98]:
df_reg.isna().sum()[34:45]

Rear Wheel Diameter      263
Rear Wheel Material       31
Rear Wheel Width         275
Rear track               554
Reverse Ratio            290
Second Gear Ratio        297
Seventh Gear Ratio      1065
Sixth Gear Ratio         359
Spare Tire Size           31
Spare Wheel Material      31
TPMS                      59
dtype: int64

In [99]:
df_reg.isna().sum()[44:]

TPMS                                                59
Third Gear Ratio                                   300
Torque                                              41
Torque RPM                                         141
Transmission                                       323
Turning circle                                     177
Weight Distributing Hitch - Max Tongue Weight     1226
Weight Distributing Hitch - Max Trailer Weight    1226
Wheelbase                                           22
Width                                               49
extra options                                      358
extra_options                                       72
price                                                0
model_name_carspecs.us                               0
third_row_seating                                    0
first_row_seating                                  261
rear_row_seating                                   452
dtype: int64

### More column drops, reinstating battery range and then saving in pickle df_reg

In [100]:
# dropping more columns
df_reg.drop(['extra options', 'Weight Distributing Hitch - Max Tongue Weight', 
             'Weight Distributing Hitch - Max Trailer Weight'], 1, inplace = True)
df_reg.drop(['ABS Brakes Type',
             'Rear Brake Rotor Diameter','Rear Drum Brake Dimensions', 'Reverse Ratio', 
             'Spare Tire Size','Spare Wheel Material', 'TPMS', 'Turning circle'], 1, inplace=True)
df_reg.drop(['0-60 mph','Front Wheel Material','Rear Brakes','Rear Wheel Material','Rear Wheel Width', 
             'Front Wheel Width'], 1, inplace=True)
df_reg.drop(['Engine Oil Cooler', 'Front Brake Rotor Diameter', 
             'Front Axle Type','Final Drive Axle Ratio', 'Max Trailering Capacity',
            'Gross Vehicle Weight Rating Cap'],1,inplace=True)
df_reg.drop(['ABS Braking System (second line)', 'Brake Type'], 1, inplace = True)

In [103]:
# number of features
len(df_reg.columns)

44

In [104]:
df_reg.columns

Index(['Base engine size', 'Curb weight', 'Cylinders',
       'Dead Weight Hitch - Max Tongue Weight',
       'Dead Weight Hitch - Max Trailer Weight', 'Drive type',
       'EPA mileage (combined)', 'Eighth Gear Ratio', 'Engine type',
       'Fifth Gear Ratio', 'First Gear Ratio', 'Fourth Gear Ratio',
       'Front Brakes', 'Front Suspension Type', 'Front Tire Size',
       'Front Wheel Diameter', 'Front track', 'Fuel tank capacity', 'Height',
       'Horsepower', 'Horsepower RPM', 'Length', 'Ninth Gear Ratio',
       'Passenger Capacity', 'Passenger Doors', 'Rear Suspension Type',
       'Rear Tire Size', 'Rear Wheel Diameter', 'Rear track',
       'Second Gear Ratio', 'Seventh Gear Ratio', 'Sixth Gear Ratio',
       'Third Gear Ratio', 'Torque', 'Torque RPM', 'Transmission', 'Wheelbase',
       'Width', 'extra_options', 'price', 'model_name_carspecs.us',
       'third_row_seating', 'first_row_seating', 'rear_row_seating'],
      dtype='object')

In [105]:
# making shire the desired features are kept
df_reg = df_reg[['model_name_carspecs.us', 'price', 'Passenger Doors','Passenger Capacity',
                 'Base engine size', 'Cylinders', 'Fuel tank capacity','EPA mileage (combined)', 'Engine type',
                 'Horsepower', 'Horsepower RPM', 'Torque', 'Torque RPM', 'Drive type','Transmission', 
             'First Gear Ratio', 'Second Gear Ratio', 'Third Gear Ratio', 
             'Fourth Gear Ratio', 'Fifth Gear Ratio', 'Sixth Gear Ratio',
             'Seventh Gear Ratio', 'Eighth Gear Ratio', 'Ninth Gear Ratio', 
             'third_row_seating', 'first_row_seating','rear_row_seating',
             'Curb weight', 'Dead Weight Hitch - Max Tongue Weight', 'Dead Weight Hitch - Max Trailer Weight', 
             'Height','Length', 'Wheelbase', 'Width', 'Front track', 'Rear track',
             'Front Brakes', 'Front Suspension Type', 'Front Tire Size','Front Wheel Diameter',
             'Rear Suspension Type', 'Rear Tire Size', 'Rear Wheel Diameter', 
             'extra_options']]

In [106]:
df_reg.columns

Index(['model_name_carspecs.us', 'price', 'Passenger Doors',
       'Passenger Capacity', 'Base engine size', 'Cylinders',
       'Fuel tank capacity', 'EPA mileage (combined)', 'Engine type',
       'Horsepower', 'Horsepower RPM', 'Torque', 'Torque RPM', 'Drive type',
       'Transmission', 'First Gear Ratio', 'Second Gear Ratio',
       'Third Gear Ratio', 'Fourth Gear Ratio', 'Fifth Gear Ratio',
       'Sixth Gear Ratio', 'Seventh Gear Ratio', 'Eighth Gear Ratio',
       'Ninth Gear Ratio', 'third_row_seating', 'first_row_seating',
       'rear_row_seating', 'Curb weight',
       'Dead Weight Hitch - Max Tongue Weight',
       'Dead Weight Hitch - Max Trailer Weight', 'Height', 'Length',
       'Wheelbase', 'Width', 'Front track', 'Rear track', 'Front Brakes',
       'Front Suspension Type', 'Front Tire Size', 'Front Wheel Diameter',
       'Rear Suspension Type', 'Rear Tire Size', 'Rear Wheel Diameter',
       'extra_options'],
      dtype='object')

In [107]:
df_reg.isna().sum()

model_name_carspecs.us                       0
price                                        0
Passenger Doors                             31
Passenger Capacity                          31
Base engine size                            66
Cylinders                                   67
Fuel tank capacity                          68
EPA mileage (combined)                     895
Engine type                                  2
Horsepower                                  32
Horsepower RPM                              66
Torque                                      41
Torque RPM                                 141
Drive type                                   0
Transmission                               323
First Gear Ratio                           275
Second Gear Ratio                          297
Third Gear Ratio                           300
Fourth Gear Ratio                          300
Fifth Gear Ratio                           306
Sixth Gear Ratio                           359
Seventh Gear 

In [112]:
# re-entering battery_range for cars here, reason is to keep the iloc index numbers for the columns
# in the earlier part of this notebook the same
df_battery = df_full.loc[~df.index.duplicated(keep='first')]
df_reg = pd.concat([df_reg, df_battery['Battery Range']],axis=1)
df_reg['Battery Range'].isna().sum()

In [115]:
pkl.dump(df_reg, open('df_pickles/df_reg.p','wb'))

2200